# Hugging face

- задачи: https://huggingface.co/tasks
- задачи для pipeline: https://huggingface.co/docs/transformers/main/en/main_classes/pipelines
- автоклассы: https://huggingface.co/docs/transformers/main/en/model_doc/auto

## Использование моделей 

### Pipeline

In [1]:
from transformers import pipeline

In [2]:
clf = pipeline(
    task='sentiment-analysis',
    model='SkolkovoInstitute/russian_toxicity_classifier')

text = ['У нас в есть убунты и текникал превью.',
    	'Как минимум два малолетних дегенерата в треде, мда.']

clf(text)

c:\Program Files\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': 'neutral', 'score': 0.9872767329216003},
 {'label': 'toxic', 'score': 0.985331654548645}]

Использование токенизатора

In [ ]:
pipeline(
    task = 'question-answering', 
    model = 'distilbert-base-cased-distilled-squad', 
    tokenizer = 'bert-base-cased')

Возврат всех классов

In [ ]:
clf(text, top_k=None)

Использование генератора для экономии памяти

In [3]:
clf = pipeline(
    task = 'sentiment-analysis', 
    model = 'SkolkovoInstitute/russian_toxicity_classifier')

text = ['У нас в есть убунты и текникал превью.',
        'Как минимум два малолетних дегенерата в треде, мда.']

def data(text):
    for row in text:
        yield row

for out in clf(data(text)):
    print(out)

{'label': 'neutral', 'score': 0.9872767329216003}
{'label': 'toxic', 'score': 0.985331654548645}


### PyTorch

In [4]:
import torch
import requests
from PIL import Image
from io import BytesIO
from transformers import AutoImageProcessor, AutoModelForImageClassification

response = requests.get(
    'https://github.com/laxmimerit/dog-cat-full-dataset/blob/master/data/train/cats/cat.10055.jpg?raw=true')
img = Image.open(BytesIO(response.content))

img_proc = AutoImageProcessor.from_pretrained(
    'google/vit-base-patch16-224')
model = AutoModelForImageClassification.from_pretrained(
    'google/vit-base-patch16-224')

inputs = img_proc(img, return_tensors='pt')

with torch.no_grad():
    logits = model(**inputs).logits

predicted_id = logits.argmax(-1).item()
predicted_label = model.config.id2label[predicted_id]
print(predicted_id, '-', predicted_label)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

281 - tabby, tabby cat


Автоматическое определение архитектуры

In [ ]:
img_proc = AutoImageProcessor.from_pretrained(
    'google/vit-base-patch16-224')
model = AutoModelForImageClassification.from_pretrained(
    'google/vit-base-patch16-224')

tokenizer = AutoTokenizer.from_pretrained(
    'SkolkovoInstitute/russian_toxicity_classifier')
model = AutoModelForSequenceClassification.from_pretrained(
    'SkolkovoInstitute/russian_toxicity_classifier')

## Дообучение

### Trainer

In [ ]:
import datasets
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)

# Загружаем данные
df = pd.read_csv('toxic.csv')
df.columns = ['text','label']
df['label'] = df['label'].astype(int)

# Конвертируем датафрейм в Dataset
train, test = train_test_split(df, test_size=0.3)
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

# Выполняем предобработку текста
tokenizer = AutoTokenizer.from_pretrained(
    'SkolkovoInstitute/russian_toxicity_classifier')

def tokenize_function(examples):
	return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_train = train.map(tokenize_function)
tokenized_test = test.map(tokenize_function)

# Загружаем предобученную модель
model = AutoModelForSequenceClassification.from_pretrained(
	'SkolkovoInstitute/russian_toxicity_classifier',
	num_labels=2)

# Задаем параметры обучения
training_args = TrainingArguments(
	output_dir = 'test_trainer_log',
	evaluation_strategy = 'epoch',
	per_device_train_batch_size = 6,
	per_device_eval_batch_size = 6,
	num_train_epochs = 5,
	report_to='none')

# Определяем как считать метрику
metric = evaluate.load('f1')
def compute_metrics(eval_pred):
	logits, labels = eval_pred
	predictions = np.argmax(logits, axis=-1)
	return metric.compute(predictions=predictions, references=labels)

# Выполняем обучение
trainer = Trainer(
	model = model,
	args = training_args,
	train_dataset = tokenized_train,
	eval_dataset = tokenized_test,
	compute_metrics = compute_metrics)

trainer.train()

# Сохраняем модель
save_directory = './pt_save_pretrained'
#tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)
#alternatively save the trainer
#trainer.save_model('CustomModels/CustomHamSpam')

# загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(
	'./pt_save_pretrained')

### PyTorch

In [ ]:
import torch
import evaluate
import pandas as pd
from tqdm.auto import tqdm
from datasets import Dataset
from torch.optim import AdamW
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, get_scheduler)

# Загружаем данные
df = pd.read_csv('toxic.csv')
df.columns = ['text','label']
df['label'] = df['label'].astype(int)

# Конвертируем датафрейм в Dataset
train, test = train_test_split(df, test_size=0.2)
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

# Выполняем предобработку текста
tokenizer = AutoTokenizer.from_pretrained(
	'SkolkovoInstitute/russian_toxicity_classifier')

def tokenize_function(examples):
	return tokenizer(examples['text'], padding='max_length', truncation=True)

def ds_preproc(ds):
	ds = ds.map(tokenize_function)
	ds = ds.remove_columns(['text', 'index_level_0'])
	ds = ds.rename_column('label', 'labels')
	ds.set_format('torch')
	return ds

tokenized_train = ds_preproc(train)
tokenized_test = ds_preproc(test)

# Создаем даталоадер
train_dataloader = DataLoader(tokenized_train, shuffle=True, batch_size=8)
test_dataloader = DataLoader(tokenized_test, batch_size=8)

# Загружаем модель и указываем кол-во классов
model = AutoModelForSequenceClassification.from_pretrained(
	'SkolkovoInstitute/russian_toxicity_classifier',
	num_labels=2)

# Задаем оптимайзер и шедулер
optimizer = AdamW(model.parameters(), lr=5e-6)

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
	name = 'linear',
	optimizer = optimizer,
	num_warmup_steps = 0,
	num_training_steps = num_training_steps)

device = 'cuda'
model.to(device)

# Выполняем цикл...
for epoch in tqdm(range(num_epochs)):

	#... обучения
	model.train()
	for batch in tqdm(train_dataloader, leave=False):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    	lr_scheduler.step()
    	optimizer.zero_grad()

	#... оценки
	metric = evaluate.load('f1')

	model.eval()
	for batch in tqdm(test_dataloader, leave=False):
    	batch = {k: v.to(device) for k, v in batch.items()}
    	with torch.no_grad():
        	outputs = model(**batch)
    	logits = outputs.logits
    	predictions = torch.argmax(logits, dim=-1)

    	metric.add_batch(predictions=predictions, references=batch['labels'])

	print(f'epoch {epoch} -', metric.compute())

# Сохраняем модель
save_directory = './pt_save_pretrained'
model.save_pretrained(save_directory)

### Вытащить эмбединги

Вручную

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
	token_embeddings = model_output[0] #First element of model_output contains all token embeddings
	input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
	sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
	sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
	return sum_embeddings / sum_mask

#Sentences we want sentence embeddings for
sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog.']

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained(
	'sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained(
	'sentence-transformers/all-MiniLM-L6-v2')

#Tokenize sentences
encoded_input = tokenizer(
	sentences, 
	padding=True, 
	truncation=True, 
	max_length=128, 
	return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
	model_output = model(encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(
	model_output,
	encoded_input['attention_mask'])

df = pd.DataFrame(sentence_embeddings).astype('float')

Через SentenceTransformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('SkolkovoInstitute/russian_toxicity_classifier')

text = ['У нас в есть убунты и текникал превью.',
    	'Как минимум два малолетних дегенерата в треде, мда.']

embeddings = model.encode(text)

df = pd.DataFrame(embeddings)

## Примеры

https://huggingface.co/docs/transformers/notebooks

https://huggingface.co/docs/transformers/community#community-notebooks

https://github.com/huggingface/transformers/tree/main/examples

https://github.com/NielsRogge/Transformers-Tutorials

Отдельно стоит отметить целый курс, посвященный трансформерам: https://huggingface.co/course/chapter1/1